In [3]:
from llama_index.tools.tavily_research import TavilyToolSpec
import os
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow

In [4]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

In [5]:
llm = Ollama("llama3.2:1b")

In [6]:
workflow = FunctionAgent(
    tools = tavily_tool.to_tool_list(),
    llm = llm,
    system_prompt = "You are a helpful assistant that can search the web for information"
)

In [7]:
from llama_index.core.agent.workflow import AgentStream

In [8]:
handler = workflow.run(user_msg = "What's the weather like in San Franscisco?")
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

In [9]:
import nest_asyncio
nest_asyncio.apply()